## Create MosaicDataset(s) and MultiDim cloud-free composite from Sentinel-2 on MS Planetary Computer - 9 timePeriod version

This version of the workflow is based on a Polygon FeatureClass defining the areas to use. 
It will cycle through this FeatureClass to create MosaicDatasets of Sentinel-2 based on MSPC.
It will then, based on these Mosaic Datasets, create Multidimensional CRFs with monthly averages of three Indixes (EVI, NDMI, MCARI-mod) as 3band timeslices OR 7bands as described in the PSETAE model.
These resulting CRFs are the basis for Training a crop detection deep Learning model using the PSETAE model.


In [ ]:
#import the modules used in this Notebook
import arcpy
from arcpy.ia import *
import os,sys
from datetime import datetime
import json
#writers decision :)
arcpy.env.overwriteOutput=True

In [ ]:
#use a little trick to get the local folder of this notebook as the reference base path
#all paths then deducted from that
from base_fns import get_local_folder
base_path = get_local_folder()
print('This notebook and all code in: {}'.format(base_path))

In [ ]:
#inputs to run this.
# there should be no need to make any changes besides in THIS field
#-----------------#

'''Important paths first - as this runs in an MDCS folder structure'''
'''------------------------------------------------------------------'''
#specific to mdcs
#as this is a custom version of the sentinel-2 workflow, special path!
#solutionLib_path=r'E:\Projects\CropDetection_PSETAE\mdcsworkflow\Sentinel2custom\scripts'
solutionLib_path=os.path.join(os.path.dirname(base_path),'mdcsworkflow\Sentinel2custom\scripts')
#this path spec is needed for the MDCS calls - please adjust if needed
pythonPath=r"C:\Program Files\ArcGIS\Pro\bin\Python\envs\arcgispro-py3\python.exe"
configBase = os.path.join(os.path.dirname(solutionLib_path), "Parameter","Config")
rftBase=os.path.join(os.path.dirname(solutionLib_path), "Parameter","RasterFunctionTemplates")
sys.path.append(solutionLib_path)
#now that ball this is set, import MDCS
import MDCS

'''Some cloud and data selection specific specific settings'''
'''---------------------------------------------------------'''
#Where is the MSPC ACS file
#acs_filepath=r"E:\ACSFiles\esrims_pc_sentinel2-l2a.acs"
acs_filepath=os.path.join(os.path.dirname(base_path),'mdcsworkflow\Sentinel2custom\Parameter\Credentials','esrims_pc_sentinel2-l2a.acs')
#max acceptable cloud_cover in % value
max_clouds=30
#The cloud we use
cloud_type="azure"
#Do we provide a list of scenes
inlist='None'
#the coordinate system we hand the extents over - we use wgs84
project_WKID=4326

'''Setting for the time period/year and time slices to use'''
'''----------------------------------------------------------'''
#the year this will work on - as a string
theyear='2023'
#define the time_slices we want to operate on - mid_month sequences from March through October
#timeranges=[[theyear+'-02-15',theyear+'-03-14',theyear+'-03-01'],[theyear+'-03-15',theyear+'-04-14',theyear+'-04-01'],
#            [theyear+'-04-15',theyear+'-05-14',theyear+'-05-01'],[theyear+'-05-15',theyear+'-06-14',theyear+'-06-01'],
#            [theyear+'-06-15',theyear+'-07-14',theyear+'-07-01'],[theyear+'-07-15',theyear+'-08-14',theyear+'-08-01'],
#            [theyear+'-08-15',theyear+'-09-14',theyear+'-09-01'],[theyear+'-09-15',theyear+'-10-14',theyear+'-10-01'],
#            [theyear+'-10-15',theyear+'-11-14',theyear+'-11-01']]
#define the time_slices we want to operate on - start/end/label 25day sequences mid from March through early Nov
timeranges=[[theyear+'-03-10',theyear+'-04-10',theyear+'-03-25'],[theyear+'-04-11',theyear+'-05-06',theyear+'-04-24'],
            [theyear+'-05-07',theyear+'-06-02',theyear+'-05-20'],[theyear+'-06-03',theyear+'-06-28',theyear+'-06-15'],
            [theyear+'-06-29',theyear+'-07-24',theyear+'-07-12'],[theyear+'-07-25',theyear+'-08-19',theyear+'-08-10'],
            [theyear+'-08-20',theyear+'-09-15',theyear+'-09-03'],[theyear+'-09-16',theyear+'-10-10',theyear+'-09-29'],
            [theyear+'-10-10',theyear+'-11-04',theyear+'-10-23']]


'''The polygon FC that defines the regions/areas for which MDs and CRFs are to be produced'''
'''------------------------------------------------------------------------------------------'''
#the regions-polygons used to define the different sub-areas
boundary_FC=os.path.join(os.path.dirname(base_path),'data\input\ATAereas\LWHPG','STATISTIK_AUSTRIA_LWHPG_20230101_plus.shp')
#the list of features and assigned names for the regions inside this FC to use - identifier is the FID
#we use shorter names instead of the real names in the FC!
#this shows [Identifier,Name] and this s the ultimate list to process
#area_list=[['1','Voralpen'],['2','Alpenostrand'],['3','Muehlviertel'],['4','Kaernten'],['5','Alpenvorland'],['6','Styria'],['7','Burgenland']]
#for partial processing and tests, just use partial lists
#there are also two test areas in the dataset:
#area_list=[['8','smallTest'],['9','largerTest']]
area_list=[['8','smallTest'],['9','LargerTest']]
#To create a buffered project boundary, specify a buffer-size for the outside buffer - 100 (m) default
buffer_size=100
#We currently dont use the MDCS feature to hand over a list of sentinel scenes as a list
in_list='None'

'''Output location and basename'''
'''-------------------------------'''
#The folder path for the fGDB where the the created Mosaics will reside
FGDB_Folder=os.path.join(os.path.dirname(base_path),'data\output\\fGDBs') 
#The fGDB name where the different Mosaics reside
FGDB_Name='AT_CropRegion_S2_MDs_2023.gdb'
#the final output-path for the Multidimensional CRFs
# be ware they are big - but rather dont save them on the ephemeral drive!
RESULT_Folder=os.path.join(os.path.dirname(base_path),'data\output\MDimRasters') 
#specify a TEMP path - on MSPC computer, recommended to use the ephemeral Z Drive
TEMP_Folder=r'z:\temp'

'''Some processing-specific settings'''
#there currently is an issue with the <GeometricMedia> function. However, in the future, once this is fixed
#this is the preferred function to use. For now we use the normal <Median> or <MEAN>. This switch will allow to change that
# set this to either 'MEAN, 'MEDIAN' or 'GEOMETRIC'
average_to_use='MEAN'#using MEAN as there are not enough rasters for a senseful median
#output_bands to produce: We try with 3 index based bands, called '3IDX' here. If that does not work, we return to the
#PSETAE recommendation of '7BND'
output_band_mode='7BND'
#the md's for the areas only have to be built once, turn the switch on here to build them
#currently they are allbuild and as they are for 2023, there wont be additional scenes available, unless cloud-% is changed
# md_mode can be 'BUILD_MD' or 'NO_BUILD_MD'
#md_mode='NO_BUILD_MD'
md_mode='BUILD_MD'

<font color='red'>DONT CHANGE ANYTHING BELOW HERE UNLESS YOU KNOW WHAT YOU ARE DOING :)</font>

In [ ]:
#all repeated sub_routines used in this Notebook#
#-----------------------------------------------#

'''Get an extent string projected into target WKID'''
'''--------------------------------------------------'''
def return_projected_feature_layer_extent(in_fc, in_query,out_WKID):
    theLyr= arcpy.MakeFeatureLayer_management(in_fc, 'myLyr',in_query)
    desc=arcpy.Describe(in_fc)
    in_sr=desc.spatialReference

    with arcpy.da.SearchCursor(theLyr,['Shape@']) as cursor:
        for row in cursor:
            e= row[0].extent
            e_points = [
                arcpy.Point(e.XMin, e.YMin),
                arcpy.Point(e.XMax, e.YMax),
                ]
            e_geometry = arcpy.Polyline(arcpy.Array(e_points), in_sr)
            out_geometry= e_geometry.projectAs(arcpy.SpatialReference(out_WKID)).extent
            out_geometry = str(out_geometry).split('NaN')[0].strip()
            arcpy.Delete_management(theLyr)
            return (out_geometry)
        
'''Create single MD - many things hardcoded'''
'''------------------------------------------'''
#this uses the MDCS workfow for Sentinel-2 on MSPC that is customized and copied into this project
#as a folder called 'mdcsworkflows'
def md_from_poly_and_MSPC(out_MD_name,out_query,out_extent):
    #import MDCS
    #NOTE: There is a custom routine for boundary cleaning and Footprint clipping used!
    cmdList="sentinel2+CM+embed_acs+AR+IG+CV+SP+bndimport"
    configName=os.path.join(configBase,'sentinel2l2A.xml')
    out_MD=os.path.join(FGDB_Folder,FGDB_Name,out_MD_name)
    startdate=timeranges[0][0]
    enddate=timeranges[-1][1]
    args = [
           '#gprun',
            f'-m:{out_MD}',
            f'-i:{configName}',
            f'-c:{cmdList}',
            f'-p:{startdate}$startdate',
            f'-p:{enddate}$enddate',
            f'-p:{out_extent}$aoi',
            f'-p:{project_WKID}$aoisrs',
            f'-p:{max_clouds}$cloudcover',
            f'-p:{cloud_type}$cloud_type',
            f'-p:{acs_filepath}$acs_filepath',
            f'-p:{in_list}$inlist',
            f'-p:{buffer_size}$buffersize',
            f'-p:{boundary_FC}$sourcefc',
            f'-p:{out_query}$sourcequery',        
            ]    

    #now ready to run
    argc = len(args)
    ret = MDCS.main(argc, args) 
    return True
    
'''Define function to remove cloud pixels based on QA band - bands hardcoded'''
'''--------------------------------------------------------------------------'''
#Also masks out anything outside the defined polygon geometry that is the boundary of the input polygon
def remove_cloud(item):#also clips to project area!
    raster = item['Raster']
    #clip to project boundary
    raster_clipped = arcpy.ia.Clip(raster, aoi = in_boundary)
    #all_bands=raster.getRasterBands()
    #for band in all_bands:
    #    print (band.name)
    #return False    
    sclband = raster.getRasterBands(['Band_15'])
    #print ('{},{},{},{}'.format(scl_band.name,scl_band.isInteger,scl_band.pixelType,scl_band.computeStatistics()))
    #first clip this mask to the project boundary
    sclband_clipped = arcpy.ia.Clip(sclband,aoi=in_boundary) 
    #now remap based on the SCL band to take out all NoData, Clouds, CloudShadow, Water, Cirrus, Undefined Pixels
    cloud_mask=arcpy.ia.Remap(raster=sclband,input_ranges=[4,6,11,15], output_values=[1,1],no_data_ranges=[0,4,6,11],allow_unmatched=False)
    #with arcpy.EnvManager(rasterStatistics="STATISTICS 128 128", resamplingMethod="BILINEAR", tileSize="512 512", pyramid="PYRAMIDS -1 NEAREST LZ77 75 SKIP_FIRST NO_SIPS", processorType="CPU", parallelProcessingFactor="80%"):
    #    arcpy.management.CopyRaster(in_raster=cloud_mask, out_rasterdataset=os.path.join(r'Z:\temp','remap_'+str(item["Name"])+'.crf'),  format='CRF') 
    #and finally clip the original raster (with all its bands) using the remapped dataset    
    cloud_free_raster = arcpy.ia.Clip(raster_clipped, aoi = cloud_mask)
    #with arcpy.EnvManager(rasterStatistics="STATISTICS 128 128", resamplingMethod="BILINEAR", tileSize="512 512", pyramid="PYRAMIDS -1 NEAREST LZ77 75 SKIP_FIRST NO_SIPS", processorType="CPU", parallelProcessingFactor="80%"):
    #    arcpy.management.CopyRaster(in_raster=cloud_free_raster, out_rasterdataset=os.path.join(r'Z:\temp','clip_'+str(item["Name"])+'.crf'),  format='CRF')
    #then return the raster back into the now cloud_free and clipped Raster collection    
    return {'raster': cloud_free_raster, "Name": item["Name"], "AcquisitionDate": item["AcquisitionDate"]}    


'''The BINARY_MASK function just creates a mask: Where there is a value, make it one, else, NoData'''
'''-------------------------------------------------------------------------------------------------'''
#this can be used to debug the availability of all time-slices in an area. NOT part of the regular workflow
#see commented out part at the end
def BINARY_MASK(item):
    # Create the raster object from the item of one of the resulting images
    #as we only care for NoData, this version should work for all types
    raster = item['Raster']
    #for 3IDX the we use the NDMI
    if output_band_mode=='3IDX':
        sel_band=raster.getRasterBands(["NDMI"])
    else:#the 7band option
        sel_band=raster.getRasterBands(["Red"])       
    binary_mask=arcpy.ia.Remap(raster=sel_band,input_ranges=[-65535,65535], output_values=[1],allow_unmatched=False)
    #if you want to debug
    #out_p=r'c:\temp\bm'+str(item['StdTime'][:10])+'.tif'
    #print (out_p)
    #arcpy.management.CopyRaster(binary_mask, out_p, config_keyword=None, background_value=None, nodata_value=0, format='COG')
    return {"raster": binary_mask, "StdTime": item['StdTime']}


'''Calculate 3 indices as three bands ... maybe add others ... ? work in progress'''
'''------------------------------------------------'''
#the definition of the indices used - work in progress - not used for now
def ThreeIndices(item):
    raster=item['Raster']
    #we need to store those away - might be too big for memory!
    outpath=os.path.join(TEMP_Folder,'threeIndixces_'+item['datestr']+'.tif')
    #get all bands used in the three indixces
    blue,green,red,rededge,nir,swir = raster.getRasterBands(["Band_2","Band_3","Band_4","Band_5","Band_8","Band_11"])
    #evi - sort of similar to ndvi - bringing to suitable 16 bit range by multiplying by 10000
    EVI = (2.5*(((nir - red) / (nir + 6*red - 7.5*blue)) + 1))*10000
    #EVI = 2.5*(((nir - red) / (nir + 6*red - 7.5*blue)) + 1)#org formula
    for bname in EVI.bandNames:
        EVI.renameBand(bname, 'EVI')
    #NDMI
    NDMI = (nir-swir)/(nir+swir)*10000 #*10000
    #NDMI = (nir-swir)/(nir+swir)#org formula
    
    for bname in NDMI.bandNames:
        NDMI.renameBand(bname, 'NDMI')
    #mcari, additionally divided by 100 to better fit the scale of EVI and NDMI
    #MCARI=(((rededge-red)-0.2*(rededge-green))*(rededge/red))/100 #org formula/100
    MCARI=(((rededge-red)-0.2*(rededge-green))*(rededge/red))*100 #org formula*100
  
    for bname in MCARI.bandNames:
        MCARI.renameBand(bname, 'MCARI')    
    #create a 3band outraster from it
    raslist=[EVI,NDMI,MCARI]
    #raslist.append(EVI)
    #raslist.append(NDMI)
    #raslist.append(MCARI)
    outras=arcpy.CompositeBands_management(raslist,outpath)  
    #print ("Little debug report: month {}, datestr {}, StdTime {}".format(item['month'],item['datestr'],item['StdTime']))
    #as the name will be used as the variable, dont create a name per raster like this ...
    #namestr=f"idx3_{item['month']}"
    #but use a constant as the name
    return {"raster": outras, "name": "idx3_composite","datestr": item['datestr'],"StdTime": item['StdTime']}

'''Return 7 Bands'''
'''------------------------------------------------'''
#limit output to 7bands
def SevenBands(item):
    raster=item['Raster']
    #we need to store those away - might be too big for memory!
    outpath=os.path.join(TEMP_Folder,'sevenBands_'+item['datestr']+'.tif')
    #get all bands needed
    blue,green,red,rededge,nir,swir1,swir2=raster.getRasterBands(["Band_2","Band_3","Band_4","Band_7","Band_8","Band_11","Band_12"])
    #create a 7band outraster from it
    raslist=[blue,green,red,rededge,nir,swir1,swir2]
    outras=arcpy.CompositeBands_management(raslist,outpath)  
    #as the name will be used as the variable, dont create a name per raster like this ...
    #namestr=f"idx3_{item['month']}"
    #but use a constant as the name
    return {"raster": outras, "name": "bnd7_composite","datestr": item['datestr'],"StdTime": item['StdTime']}

'''The routine that takes the output MDIM-Raster and creates a single
polygon from it that delineates the area of the image that has data for every single time-slice'''
def create_full_coverage_polygon(in_mdim_raster_path,item_name,in_year):
    time_reporter('Create full coverage mask and polygon')
    rc_for_remap=arcpy.ia.RasterCollection(in_mdim_raster_path)
    slice_count=int(rc_for_remap.count)
    print (slice_count)
    print ('New Collection has {} images'.format(slice_count))
    print ('Fields in collection: {}'.format(rc_for_remap.fields))
    rc_remapped=rc_for_remap.map(BINARY_MASK)
    #clean up
    del rc_for_remap
    #now summarize and remap the result again
    rc_sum=rc_remapped.sum()  
    #clean up
    del rc_remapped
    #here you can decide how tolerable to be. ALL TimeSlices would be only accepting pixels with value 7
    #for succesful export of training samples this is needed
    rc_mask=arcpy.ia.Remap(raster=rc_sum,input_ranges=[slice_count,slice_count+1], output_values=[1],no_data_ranges=[0,slice_count,slice_count+1,slice_count+10],allow_unmatched=False)
    #rc_mask=arcpy.ia.Remap(raster=rc_sum,input_ranges=[9,10], output_values=[1],no_data_ranges=[0,6,8,10],allow_unmatched=False)
    #clean up
    del rc_sum
    rc_mask_outpath=os.path.normpath(os.path.join(TEMP_Folder,item_name+'_coverage_mask.tif'))
    #arcpy.management.CopyRaster(rc_mask, rc_mask_outpath, nodata_value=0, pixel_type="8_BIT_UNSIGNED", scale_pixel_value="ScalePixelValue", format='COG')
    arcpy.management.CopyRaster(rc_mask, rc_mask_outpath, nodata_value=0, pixel_type="8_BIT_UNSIGNED", format='COG')
    #clean up
    del rc_mask
    #now make this raster to Polygon - in Temp-Space, as we'll buffer inside later
    out_temp_poly=os.path.join(FGDB_Folder,FGDB_Name,item_name+'_tempArea')
    #out_temp_poly=r'memory\temp_'+item_name+'_allTimeslices'
    arcpy.conversion.RasterToPolygon(
    in_raster=rc_mask_outpath,
    out_polygon_features=out_temp_poly,
    simplify="SIMPLIFY",
    raster_field="Value",
    create_multipart_features="MULTIPLE_OUTER_PART",
    max_vertices_per_feature=None
    )
    #now define and write the final polygon feature class
    out_final_fc=os.path.join(FGDB_Folder,FGDB_Name,item_name+'_'+in_year+'_AllTimeSlicesArea')
    arcpy.analysis.Buffer(
        in_features=out_temp_poly,
        out_feature_class=out_final_fc,
        buffer_distance_or_field="-100 Meters",
        line_side="FULL",
        line_end_type="ROUND",
        dissolve_option="ALL",
        dissolve_field=None,
        method="PLANAR"
        )
    #another final cleanup
    try:
        arcpy.Delete_management(rc_mask_outpath)
        arcpy.Delete_management(out_temp_poly)
    except Exception as exp:
        print ('delete error {}'.format(exp))
        pass
    print ('Final Coverage FC written to: {}'.format(out_final_fc))
    return None



'''attach rft-template to an existing raster (not MD)'''
'''----------------------------------------------------'''
#depending on output, define templates here and attach
def append_rfts_to_raster(in_ras):
    #if not a raster but a string is handed over, make it the raster we need
    if 'str' in str(type(in_ras)): #make it a raster
        mdim_raster=arcpy.Raster(in_ras)
        
    else: #it likely is a raster already
        mdim_raster=in_ras  
    rftlist=[]
    if output_band_mode=='3IDX':
        rftlist.append(os.path.normpath(os.path.join(rftBase,'As3Band.rft.xml')))
        rftlist.append(os.path.normpath(os.path.join(rftBase,'ExtractEVI.rft.xml')))
        rftlist.append(os.path.normpath(os.path.join(rftBase,'ExtractNDMI.rft.xml')))
        rftlist.append(os.path.normpath(os.path.join(rftBase,'ExtractMCARI.rft.xml'))) 
    else: #the 7 band usecase, use all 7bands as default, and add 3 visual bands with DRA
        rftlist.append(os.path.normpath(os.path.join(rftBase,'All7Bands.rft.xml')))       
        rftlist.append(os.path.normpath(os.path.join(rftBase,'SimpleRGB_DRA.rft.xml')))
    #now mod the raster
    for rft in rftlist:
        if mdim_raster.functions is None:
            flist=[]
            flist.append(rft)
        else:
            flist=mdim_raster.functions
            flist.append(rft)
        mdim_raster.functions=flist
    try:
        mdim_raster.save(mdim_raster.catalogPath)
    except:
        pass
    return None   


'''Reassigning the Band names by the selected band output option'''
'''-----------------------------'''
def reassign_band_names(in_ras):
    #if not a raster but a string is handed over, make it the raster we need
    if 'str' in str(type(in_ras)): #make it a raster
        mdim_raster=arcpy.Raster(in_ras)
    else: #it likely is a raster already
        mdim_raster=in_ras
    if output_band_mode=='3IDX':
        try:
            mdim_raster.renameBand('Band_1', 'EVI')
            mdim_raster.renameBand('Band_2', 'NDMI')
            mdim_raster.renameBand('Band_3', 'MCARI')
        except:
            pass
    else: #the 7 band result  blue,green,red,rededge,nir,swir1,swir2    
        try:
            mdim_raster.renameBand('Band_1', 'Blue')
            mdim_raster.renameBand('Band_2', 'Green')
            mdim_raster.renameBand('Band_3', 'Red')
            mdim_raster.renameBand('Band_4', 'Red Edge 3')
            mdim_raster.renameBand('Band_5', 'Nir')
            mdim_raster.renameBand('Band_6', 'Swir 1')
            mdim_raster.renameBand('Band_7', 'Swir 2')
        except Exception as exp:
            print (exp)
            pass  
    return None

'''A little time reporter'''
'''-----------------------'''
#just keeping code shorter
def time_reporter(in_topic_string):
    now = datetime.now()
    current_time = now.strftime("%H:%M:%S")
    print("{} started at: {}".format(in_topic_string, current_time))
    return None        

In [ ]:
#build all the mosaics - this just needs to be done once, thats why it is taken out of the larger routine below
for item in area_list:
    the_query='FID = '+item[0]
    the_name=item[1]+'_'+theyear
    the_ext=return_projected_feature_layer_extent(boundary_FC, the_query,project_WKID)
    if md_mode=='NO_BUILD_MD':
        print ('Listing MD {}: {}, Extent is {}'.format(the_name,the_query,the_ext))
    else: #BUILD MODE    
        print ('Building MD ... {}: {}, Extent is {}'.format(the_name,the_query,the_ext))
        md_from_poly_and_MSPC(the_name,the_query,the_ext)   

<font color='orange'>!!!BELOW IS THE MAIN LOOP - THIS CAN/WILL TAKE CONSIDERABLE TIME, HOURS! AND MORE TO COMPLETE!!!</font>

In [ ]:
%%time
# THE MAIN ROUTINE LOOP
#we build the analytics on top of an existing Mosaic Datasets, 
# pre-requisit is thus having run the code-block above to create the Mosaics
# based on the area_list we now cycle though the Mosaics and do all the work ...
#arcpy.Delete_management(in_md_layer)
#arcpy.Delete_management('in_MDLayer')
for item in area_list:
    input_md=os.path.join(FGDB_Folder,FGDB_Name,item[1]+'_'+theyear)
    print('-----------------------------------------------')
    print ('Working on MosaicDataset {}'.format(input_md))
    print('-----------------------------------------------')
    time_reporter('Process main loop')
    #now get the polygon of the boundary right away to use for clipping
    in_md_layer = arcpy.MakeMosaicLayer_management(input_md, 'in_MDLayer')
    in_boundary = 'in_MDLayer\Boundary'
    #create the raster collection and report on it
    my_col=arcpy.ia.RasterCollection(input_md)
    print ('Collection has {} images'.format(my_col.count))
    print ('Fields in collection: {}'.format(my_col.fields))
    #split the rc up into the monthly pieces and create averaged corrected versions per time period
    monthly_rasters=[]
    monthly_dates=[]
    #cycle through all time_ranges
    print ('-------------------------------')
    print ('Looping through timeslices')
    for timerange in timeranges:
        #mid_date=timerange[1][:-2]+'01'
        mid_date=timerange[2]
        #filter by time period
        #STEP 1: THE MONTHLY SLICES
        #--------------------------
        rc_month=my_col.filterByTime (timerange[0], timerange[1], 'AcquisitionDate', date_time_format = '%Y-%m-%d')
        try:
            print ('Working on {} with: {} rasters'.format(mid_date,rc_month.count))
            #first remove the clouds
            time_reporter('Mapping cloud removal function')
            month_cloudfree=rc_month.map(remove_cloud)
            #now we need to decide on which median is used
            if average_to_use=='MEDIAN':
                time_reporter('Simple MEDIAN calculation')
                try:
                    month_acc = month_cloudfree.median(ignore_nodata = True, extent_type = 'UnionOf')
                    month_acc_output=os.path.join(TEMP_Folder,'acc_'+str(rc_month.count)+'_median_'+mid_date+'.tif')
                    #always output as 16bit integer!
                    if output_band_mode=='3IDX':
                        arcpy.management.CopyRaster(month_acc, month_acc_output, config_keyword=None, pixel_type='16_BIT_UNSIGNED', background_value=None, nodata_value=0, format='COG')
                        #month_acc.save(month_acc_output)
                    else:
                        arcpy.management.CopyRaster(month_acc, month_acc_output, config_keyword=None, pixel_type='16_BIT_UNSIGNED', background_value=None, nodata_value=0, format='COG')
                except Exception as exp:
                    print (exp)
            elif average_to_use=='MEAN':        
                time_reporter('Simple MEAN calculation')
                try:
                    month_acc = month_cloudfree.mean(ignore_nodata = True, extent_type = 'UnionOf')
                    month_acc_output=os.path.join(TEMP_Folder,'acc_'+str(rc_month.count)+'_mean_'+mid_date+'.tif')
                    if output_band_mode=='3IDX':
                        #month_acc.save(month_acc_output)
                        arcpy.management.CopyRaster(month_acc, month_acc_output, config_keyword=None, pixel_type='16_BIT_UNSIGNED', background_value=None, nodata_value=0, format='COG')
                    else:
                        arcpy.management.CopyRaster(month_acc, month_acc_output, config_keyword=None, pixel_type='16_BIT_UNSIGNED', background_value=None, nodata_value=0, format='COG')
                except Exception as exp:
                    print (exp)

            else: #thats the GEOMETRIC MEDIAN that does not work porperly - we also have to use intermediate files more
                time_reporter('GEOMETRIC MEDIAN calculation')
                try:
                    #we need a physical file here - fails if trying to do in memory
                    temp_multidim=month_cloudfree.toMultidimensionalRaster(variable_field_name = "Name", dimension_field_names = "AcquisitionDate")
                    cloudfree_outpath=os.path.normpath(os.path.join(TEMP_Folder,'cloudfree_'+str(rc_month.count)+'_'+mid_date+'.crf'))
                    temp_multidim.save(cloudfree_outpath)
                    #now we can continue similar to above
                    month_acc=GeometricMedian(temp_multidim.catalogPath, epsilon=25, max_iteration=rc_month.count-2, extent_type='UnionOf', cellsize_type='FirstOf')
                    month_acc_output=os.path.join(TEMP_Folder,'acc_'+str(rc_month.count)+'_geometric_'+mid_date+'.tif')
                    #adjust the pixel type depending on mode
                    if output_band_mode=='3IDX':
                        arcpy.management.CopyRaster(month_acc, month_acc_output, config_keyword=None, pixel_type='16_BIT_UNSIGNED', background_value=None, nodata_value=0, format='COG')
                    else:
                        arcpy.management.CopyRaster(month_acc, month_acc_output, config_keyword=None, pixel_type='16_BIT_UNSIGNED', background_value=None, nodata_value=0, format='COG')
 
                    arcpy.management.CopyRaster(month_acc, month_acc_output, config_keyword=None, pixel_type='16_BIT_UNSIGNED', background_value=None, nodata_value=0, format='COG')
                except Exception as exp:
                    print (exp)   
            #now add the result to the respective lists
            monthly_rasters.append(month_acc_output)
            monthly_dates.append(mid_date)
            #and we can do some clean_up
            #we dont need month_cloudfree anymore, delete
            del month_cloudfree
            #there is more we dont need if the version was GEOMETRIC MEDIAN
            if average_to_use=='GEOMETRIC':
                #we dont need the cloudfree stored version anymore
                del temp_multidim
                arcpy.Delete_management(cloudfree_outpath)
        except Exception as exp:
            print ('Error during monthly slice generation/nDetails {}'.format(exp))
        print ('Finished processing Slice {}'.format(mid_date))    
    #STEP 2: RE-CREATE A RASTER COLLECTION FROM THE MONTHLY AVERAGED SLICES
    #----------------------------------------------------
    print ('-------------------------------')
    time_reporter('Create new Multidim raster')
    try:
        the_dict={'StdTime':monthly_dates,'datestr':monthly_dates}
        monthly_rc=arcpy.ia.RasterCollection(monthly_rasters,the_dict)
        print ('New Collection has {} images'.format(monthly_rc.count))
        print ('Fields in collection: {}'.format(monthly_rc.fields))
        #apply the band combination/selection desired
        if output_band_mode=='3IDX':
            outband_rc=monthly_rc.map(ThreeIndices)
            mdim_outpath=os.path.join(RESULT_Folder,os.path.basename(input_md)+'_3idx.crf')
            print (mdim_outpath)
        else: #the 7 band mode
            outband_rc=monthly_rc.map(SevenBands)
            mdim_outpath=os.path.join(RESULT_Folder,os.path.basename(input_md)+'_7bnd.crf')           
        #and save the output plus do cleanup
        time_reporter('Safe final output')
        mdim_raster = outband_rc.toMultidimensionalRaster(variable_field_name = "name", dimension_field_names = 'StdTime')
        mdim_raster.save(mdim_outpath)

        #there are locking issues with mdim_raster - clean up temp datasets and variables
        #work with the path for the following steps
        try:
            del mdim_raster
            del outband_rc
            del monthly_rc
            del in_md_Layer
            del in_boundary
            arcpy.Delete_management('in_MDLayer')
        except Exception as exp:
            print (exp)
            pass
            
        #reassign the bands again :(
        time_reporter('Re-assigning bands')
        reassign_band_names(mdim_outpath)

        #build the multidimensional transpose on it
        time_reporter('Multidimensional transpose')
        arcpy.md.BuildMultidimensionalTranspose(
            in_multidimensional_raster=mdim_outpath,
            delete_transpose="NO_DELETE_TRANSPOSE")

        #apply predefined rfts to the result
        time_reporter('Append rfts')
        append_rfts_to_raster(mdim_outpath)

    except Exception as exp:
        print ('Error during re-creation of Multidim raster/Details {}'.format(exp))
            
    #STEP 3 - little cleanup of written temp files
    #---------------------------------------------
    time_reporter('Cleanup')
    arcpy.env.workspace=TEMP_Folder
    try:
        for dataset in arcpy.ListRasters('acc*.tif'):
            fullpath=os.path.join(arcpy.env.workspace,dataset)
            arcpy.Delete_management(fullpath)
        #take care of output_band_mode selected
        if output_band_mode=='3IDX':
            for dataset in arcpy.ListRasters('three*.tif'):
                fullpath=os.path.join(arcpy.env.workspace,dataset)
                arcpy.Delete_management(fullpath)  
        else: #the 7band mode
            for dataset in arcpy.ListRasters('seven*.tif'):
                fullpath=os.path.join(arcpy.env.workspace,dataset)
                arcpy.Delete_management(fullpath)  
            
    except Exception as exp:
        print ('Error during cleanup/Details {}'.format(exp))
                                
    print ('Finished processing MD {}'.format(input_md))
    print ('Final result written to {}'.format(mdim_outpath))
    print ('----------------------------------------')                            

    #STEP 4: - create a single polygon per result, that delineates
    #------------------------------------------------------------
    #the part of the result that has valid pixels for every time-slice
    #this is needed for the training sample generation
    #this polygon will be saved in the same fGDB as the original mosac
    create_full_coverage_polygon(mdim_outpath,item[1],theyear)        
print ('All done.')            

In [ ]:
arcpy.Delete_management('in_MDLayer')